In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from rich.pretty import pprint
from rich.markdown import Markdown

load_dotenv()


MODEL = os.getenv("MODEL", "gemini-2.5-flash-lite")
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
llm = ChatOpenAI(
    model=MODEL,
    base_url=BASE_URL,
    api_key=API_KEY,  # type: ignore
    temperature=0,
)

print(llm.invoke("Hi!").content)

Hi there! How can I help you today?


In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage

In [32]:
from typing import TypedDict, Annotated, NotRequired
import operator

class AgentState(TypedDict):
    topic: str
    jokes: NotRequired[Annotated[list[str], operator.add]]
    final_joke: NotRequired[str]
    # poem: NotRequired[str]
    # joke: NotRequired[str]
    # story: NotRequired[str]

In [ ]:
def _generate_joke(state: AgentState):
    joke = llm.invoke([HumanMessage(
        content=f"Tell me a joke about {state['topic']}"
    )])
    return {"joke": joke.content}

def _generate_poem(state: AgentState):
    poem = llm.invoke([HumanMessage(
        content=f"Write a poem about {state['topic']}"
    )])
    return {"poem": poem.content}

def _generate_story(state: AgentState):
    story = llm.invoke([HumanMessage(
        content=f"Write a short story about {state['topic']}"
    )])
    return {"story": story.content}

In [27]:
graph = StateGraph(AgentState)

graph.add_node("poem", _generate_poem)
graph.add_node("joke", _generate_joke)
graph.add_node("story", _generate_story)

graph.add_edge(START, "poem")
graph.add_edge(START, "joke")
graph.add_edge(START, "story")

agent = graph.compile()
# agent
print(agent.get_graph().draw_ascii())

              +-----------+                
              | __start__ |                
              +-----------+*               
            **      *       **             
         ***        *         ***          
       **           *            **        
+------+        +------+        +-------+  
| joke |*       | poem |        | story |  
+------+ ***    +------+      **+-------+  
            **      *       **             
              ***   *    ***               
                 ** *  **                  
               +---------+                 
               | __end__ |                 
               +---------+                 


In [15]:
response = agent.invoke({"topic":"monsoon season"})

In [16]:
print(f"Poem: {response['poem']}\n")
print(f"Joke: {response['joke']}\n")
print(f"Story: {response['story']}\n")

Poem: The sky, a canvas, bruised and grey,
Holds its breath, then starts to sway.
A whisper first, a gentle sigh,
As clouds like giants drift on high.

Then, a rumble, deep and low,
A promise of the coming flow.
The air grows thick, a humid kiss,
Anticipating nature's bliss.

And then it breaks, a sudden roar,
The heavens open, and pour, and pour!
A silver curtain, thick and vast,
The world transformed, the die is cast.

The parched earth drinks with thirsty greed,
Each droplet planting life's new seed.
The dust is washed, the air is clean,
A vibrant, verdant, emerald scene.

The leaves unfurl, a brighter green,
The flowers bloom, a joyful queen.
The rivers swell, a rushing might,
Reflecting skies of stormy light.

The scent of petrichor, so sweet,
A fragrant balm for weary feet.
The rhythm of the falling rain,
A lullaby to ease all pain.

Children splash in puddles deep,
While weary souls find peaceful sleep.
The world slows down, a gentle pause,
Beneath the monsoon's ancient laws.

T

In [33]:
def _gen_joke(state: AgentState):
    joke = llm.invoke([HumanMessage(
        content=f"Tell me a joke about {state['topic']}"
    )])

    return {"jokes": [joke.content]}

In [34]:
graph = StateGraph(AgentState)

graph.add_node("joke_1", _gen_joke)
graph.add_node("joke_2", _gen_joke)
graph.add_node("joke_3", _gen_joke)
graph.add_node("joke_4", _gen_joke)

graph.add_edge(START, "joke_1")
graph.add_edge(START, "joke_2")
graph.add_edge(START, "joke_3")
graph.add_edge(START, "joke_4")

agent = graph.compile()
print(agent.get_graph().draw_ascii())

                            +-----------+                              
                            | __start__ |**                            
                       *****+-----------+  *****                       
                  *****       *         *       *****                  
             *****          **           **          *****             
          ***              *               *              ***          
+--------+          +--------+          +--------+          +--------+ 
| joke_1 |***       | joke_2 |          | joke_3 |        **| joke_4 | 
+--------+   *****  +--------+          +--------+   *****  +--------+ 
                  *****       *         *       *****                  
                       *****   **     **   *****                       
                            ***  *   *  ***                            
                              +---------+                              
                              | __end__ |                       

In [35]:
response = agent.invoke({"topic": "monsoon"})

In [38]:
len(response['jokes'])

4